In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

sys.path.append("../")
from utils.io import read_parameters
from src.plot_trajectories import normalize_speed
import networkx as nx
from griottes import get_cell_properties, generate_delaunay_graph, generate_geometric_graph, plot_2D


cmap = matplotlib.colormaps["seismic_r"]

In [ ]:
# read parameters and key file

parameter_file  = "../local/parameters.yml"
parameters = read_parameters(parameter_file)       

key_file_path = parameters["key_file"]
key_file = pd.read_csv(key_file_path)

print("key file columns:", key_file.columns)
print("key file conditions:", key_file["condition"].unique())

In [ ]:
# create output folder

output_folder = parameters["output_folder"]
print("Output folder:", output_folder)

data_folder = output_folder + "/tracking_data/"

subfolder = "velocity_field"
if not os.path.exists(output_folder + "/" + subfolder):
    os.mkdir(output_folder + "/" + subfolder)

#tracking_data = pd.read_csv(data_folder + "tracking_data.csv") 


In [ ]:
# plot paramters

condition = 'mosaic_siCTRL_siAlk1'
# color = 'red'
treatment = 'siAlk1'

#number_of_tracks_per_condition = 1000

observation_time = parameters["observation_time"]

# limits for min velocity and max velocity in um/h used for the color map
min_vel_lim = -20.0 # um/h
max_vel_lim = 20.0 # um/h

# extend in microns of the coordinate system used for plotting, 
# range is [-max_x, max_x] in x direction and [-max_y, max_y] in y direction
max_x = 600 # um
max_y = 600 # um

plot_tracks = True
plot_arrows = True

distance_threshold = 50 # um

In [ ]:
key_select = key_file[key_file["condition"] == condition]

print(key_select)

obs_time_length_frames = observation_time[1] - observation_time[0]

#for index, row in key_select.iterrows():
for experimentID in key_select["experimentID"].unique():
   
    key_exp = key_select[key_select["experimentID"] == experimentID]

    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].set_aspect('equal', 'box')
    ax[0].set_xlim(0, max_x)
    ax[0].set_ylim(0, max_y)

    ax[1].set_aspect('equal', 'box')
    ax[1].set_xlim(0, max_x)
    ax[1].set_ylim(0, max_y)

    ax[2].set_aspect('equal', 'box')
    ax[2].set_xlim(0, max_x)
    ax[2].set_ylim(0, max_y)

    plot_nr = 0
    vel_index = 0

    velocity_df = pd.DataFrame(columns=["TRACK_ID", "VEL", "VEL_X", "VEL_Y", "DELTA_X", "DELTA_Y", 
                               "START_X", "START_Y", "END_X", "END_Y"])

    for index, row in key_exp.iterrows():

        treatment = row["treatment"]

        tracking_file = "tracking_data_%s_%s_%s.csv" % (treatment, row["color"], row["experimentID"])
                    
        data = pd.read_csv(data_folder+ tracking_file, low_memory=False)

        print(data["POSITION_X"].min(), data["POSITION_X"].max())

        observation_period_df = data[data["FRAME"] <= observation_time[1]]
        observation_period_df = observation_period_df[observation_period_df["FRAME"] >= observation_time[0]]

        # Get the first frame of each track
        tracks_start = observation_period_df[observation_period_df["FRAME"] == observation_time[0]]
        
        # determine the min end point of the tracks
        # in case "allow_tracks_shorter_than_observation_time" in the parameters file is set to True
        # -> the min required track length is the min of 'obs_time_length_frames' and 'min_track_length'
        # in case "allow_tracks_shorter_than_observation_time" in the parameters file is set to False
        # -> the end point of each trajectory is the end of the observation time 

        frame_end_point = observation_time[1]
        if parameters["allow_tracks_shorter_than_observation_time"] == True:
            if obs_time_length_frames > parameters["min_track_length"]:
                frame_end_point = observation_time[0] + parameters["min_track_length"]

        # Assign tracks_end using the determined frame
        tracks_end = observation_period_df[observation_period_df["FRAME"] == frame_end_point]

    
        ax[plot_nr].set_title("Experiment, %s, %s" % (row["experimentID"], row["treatment"]))
       



        for trackID in tracks_start["TRACK_ID"].unique():

            single_track_df = observation_period_df[observation_period_df["TRACK_ID"] == trackID]
            start_frame = single_track_df["FRAME"].min()
            end_frame = single_track_df["FRAME"].max()
            delta_frame = end_frame - start_frame

            delta_hour = delta_frame/parameters["frames_per_hour"]

            row_start = single_track_df[single_track_df["FRAME"] == start_frame]
            row_end = single_track_df[single_track_df["FRAME"] == end_frame]

            start_x = np.array(row_start["POSITION_X"])[0]
            start_y = np.array(row_start["POSITION_Y"])[0]
            end_x = np.array(row_end["POSITION_X"])[0]
            end_y = np.array(row_end["POSITION_Y"])[0]
            delta_x = end_x - start_x
            delta_y = end_y - start_y
            delta = np.sqrt(delta_x**2 + delta_y**2)


            rel_vel = normalize_speed(delta_x/delta_hour, min_vel_lim, max_vel_lim)

            if plot_tracks:
                
                ax[plot_nr].plot(single_track_df["POSITION_X"], single_track_df["POSITION_Y"], color='black', alpha=0.5)
                ax[2].plot(single_track_df["POSITION_X"], single_track_df["POSITION_Y"], color=row["color"], alpha=0.5)
        
            if plot_arrows:
                ax[plot_nr].arrow(start_x, start_y, delta_x, delta_y, head_width=10, head_length=10, fc='black', ec='black')
                ax[2].arrow(start_x, start_y, delta_x, delta_y, head_width=10, head_length=10, fc=row["color"], ec=row["color"])

            velocity_df.at[vel_index, "TRACK_ID"] = trackID
            velocity_df.at[vel_index, "label"] = str(trackID) + "_" + row["treatment"]
            velocity_df.at[vel_index, "color"] = row["color"]
            velocity_df.at[vel_index, "VEL"] = delta/delta_hour
            velocity_df.at[vel_index, "VEL_X"] = delta_x/delta_hour
            velocity_df.at[vel_index, "VEL_Y"] = delta_y/delta_hour
            velocity_df.at[vel_index, "DELTA_X"] = delta_x    
            velocity_df.at[vel_index, "DELTA_Y"] = delta_y
            velocity_df.at[vel_index, "x"] = start_x
            velocity_df.at[vel_index, "y"] = start_y
            velocity_df.at[vel_index, "END_X"] = end_x
            velocity_df.at[vel_index, "END_Y"] = end_y
            
            vel_index += 1
        
        plot_nr += 1

    descriptors = velocity_df.columns

    print("Generating graph...")
    G_delaunay = generate_delaunay_graph(velocity_df[descriptors],
                            descriptors = descriptors,
                            distance=distance_threshold,
                            image_is_2D = True)
        
    print("Graph generated!")

    red_neighbors_count = {}
    green_neighbors_count = {}
    for a, d in G_delaunay.nodes(data=True):
        #print(a)
        #print(d)

        neighbors = G_delaunay[a]
        # Count the neighbors with color red
        red_count = sum(1 for neighbor in neighbors if G_delaunay.nodes[neighbor].get('color') == 'red')
        green_count = sum(1 for neighbor in neighbors if G_delaunay.nodes[neighbor].get('color') == 'green')
        # Store the count in the dictionary
        red_neighbors_count[a] = red_count
        green_neighbors_count[a] = green_count
    
    print(red_neighbors_count)
    print(green_neighbors_count)

    plot_2D(G_delaunay,
            alpha_line = 0.6,
            scatterpoint_size = 8,
            legend = True,
            legend_fontsize = 14,
            figsize = (7,7))

    plt.tight_layout()



    


    plt.show()
    plt.close()


